In [ ]:
! pip install -U git+https://github.com/IINemo/isanlp.git@discourse

In [1]:
from isanlp import PipelineCommon
from isanlp.processor_remote import ProcessorRemote
from isanlp.ru.processor_mystem import ProcessorMystem
from isanlp.ru.converter_mystem_to_ud import ConverterMystemToUd

SERVER = ''  # put the address here
address_syntax = (SERVER, 3334)
address_rst = (SERVER, 3345)

ppl = PipelineCommon([
    (ProcessorRemote(address_syntax[0], address_syntax[1], '0'),
     ['text'],
     {'sentences': 'sentences',
      'tokens': 'tokens',
      'lemma': 'lemma',
      'syntax_dep_tree': 'syntax_dep_tree',
      'postag': 'ud_postag'}),
    (ProcessorMystem(delay_init=False),
     ['tokens', 'sentences'],
     {'postag': 'postag'}),
    (ConverterMystemToUd(),
     ['postag'],
     {'morph': 'morph',
      'postag': 'postag'}),
    (ProcessorRemote(address_rst[0], address_rst[1], 'default'),
     ['text', 'tokens', 'sentences', 'postag', 'morph', 'lemma', 'syntax_dep_tree'],
     {'rst': 'rst'})
])

In [2]:
text = (
"Новости о грядущей эмиссии в США обвалили доллар и подняли цену доллара. При этом рост количества "
"долларов пока не зафиксирован. Со швейцарским франком ситуация противоположная. Стало известно , ч"
"то в феврале денежная масса Швейцарии увеличилась на 3.5%, однако биржевой курс франка и его покуп"
"ательная способность за неделю выросли."
)

In [3]:
%%time

result = ppl(text)

CPU times: user 25.5 ms, sys: 1.78 ms, total: 27.3 ms
Wall time: 10.5 s


In [4]:
result['rst']

In [5]:
print(result['rst'][0])

id: 10
text: Новости о грядущей эмиссии в США обвалили доллар и подняли цену доллара. При этом рост количества долларов пока не зафиксирован. Со швейцарским франком ситуация противоположная. Стало известно , что в феврале денежная масса Швейцарии увеличилась на 3.5%, однако биржевой курс франка и его покупательная способность за неделю выросли.
proba: 0.36902757009731957
relation: sequence
nuclearity: NN
left: Новости о грядущей эмиссии в США обвалили доллар и подняли цену доллара.
right: При этом рост количества долларов пока не зафиксирован. Со швейцарским франком ситуация противоположная. Стало известно , что в феврале денежная масса Швейцарии увеличилась на 3.5%, однако биржевой курс франка и его покупательная способность за неделю выросли.
start: 0
end: 333


In [6]:
vars(result['rst'][0])

{'id': 10,
 'left': <isanlp.annotation_rst.DiscourseUnit at 0x7f833defa0d0>,
 'right': <isanlp.annotation_rst.DiscourseUnit at 0x7f833defa510>,
 'relation': 'sequence',
 'nuclearity': 'NN',
 'proba': 0.36902757009731957,
 'start': 0,
 'end': 333,
 'text': 'Новости о грядущей эмиссии в США обвалили доллар и подняли цену доллара. При этом рост количества долларов пока не зафиксирован. Со швейцарским франком ситуация противоположная. Стало известно , что в феврале денежная масса Швейцарии увеличилась на 3.5%, однако биржевой курс франка и его покупательная способность за неделю выросли.'}

In [7]:
def extr_pairs(tree, text):
    pp = []
    if tree.left:
        pp.append([text[tree.left.start:tree.left.end], text[tree.right.start:tree.right.end], tree.relation, tree.nuclearity])
        pp += extr_pairs(tree.left, text)
        pp += extr_pairs(tree.right, text)
    return pp

In [8]:
extr_pairs(result['rst'][0], result['text'])

[['Новости о грядущей эмиссии в США обвалили доллар и подняли цену доллара.',
  'При этом рост количества долларов пока не зафиксирован. Со швейцарским франком ситуация противоположная. Стало известно , что в феврале денежная масса Швейцарии увеличилась на 3.5%, однако биржевой курс франка и его покупательная способность за неделю выросли.',
  'sequence',
  'NN'],
 ['Новости о грядущей эмиссии в США обвалили доллар',
  'и подняли цену доллара.',
  'joint',
  'NN'],
 ['При этом рост количества долларов пока не зафиксирован.',
  'Со швейцарским франком ситуация противоположная. Стало известно , что в феврале денежная масса Швейцарии увеличилась на 3.5%, однако биржевой курс франка и его покупательная способность за неделю выросли.',
  'comparison',
  'NN'],
 ['Со швейцарским франком ситуация противоположная.',
  'Стало известно , что в феврале денежная масса Швейцарии увеличилась на 3.5%, однако биржевой курс франка и его покупательная способность за неделю выросли.',
  'evidence',
  'NS

In [9]:
import sys
sys.path.append('../')
sys.path.append('../../')

from _isanlp_rst.src.isanlp_rst.export.to_rs3 import ForestExporter

exporter = ForestExporter(encoding='utf8')
exporter(result['rst'], 'example.rs3')

Output:

![example.rs3](example.rs3.png)